In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [19]:
df = pd.read_csv('data/Labelled Yelp Dataset Edited.csv')
target = len(df.loc[df['Label'] == -1])
pd.set_option ('display.max_colwidth', None)            

In [42]:
print(f'''Number of targets: {target}
Percent of data target: {round(target / len(df),3)*100}
Number of rows: {len(df)}''')

Number of targets: 11154
Percent of data target: 10.0
Number of rows: 111798


There are 11154 fake reviews that make up 10 percent of the data. If we were to go ahead and classify everything as a real review, there would be a 90% accuracy rate. 

In [21]:
df.isna().sum()

User_id       0
Product_id    0
Rating        0
Date          0
Review        0
Label         0
dtype: int64

There are no null values in our data set.

In [40]:
df.loc[df['Label'] == -1].tail(20)

,User_id,Product_id,Rating,Date,Review,Label
111331,69314,54,5,5/6/2013,"The best restaraunt in New York City and one of the most historical places. I come here a lot. I love to support them instead of supporting other restaraunts in manhattan. It's my favorite, great food, great service, and most of all its a revolutionary war place which is my favorite! I can't believe George Washington slept, ate, and stayed here during the 1700s ! Bowling green is nearby which is a nice historical park, as well as other revolutionary war sites you should, visit. I live nearby on Chambers St and come here a lot to support them. Please support them the. Money goes to great cause. Museum is the best upstairs too.",-1
111332,69315,54,5,3/31/2013,"The bar is such a great spot and has so much character! Over 100 some beers from $6-$12 bucks, Â and the food is great too. The shepherds pie is AMAZING and comes in a pan!!",-1
111333,69316,54,5,1/2/2013,The best place to have a beer and a bite. They have hundreds of different kinds of beers.,-1
111334,69317,54,5,11/6/2011,I never even heard of this place before signing up for the Zombies vs Vampires Halloween Scavenger Hunt but now that I know about I will be back! First of all it's like a restaurant and brew pub and sports bar and museum all wrapped into 1. The place is HUGE! The prices are cheap and there's enough room so you're not super crowded or packed in. Before the scavenger hunt I was in the back bat watching the series game and having a beer. The bartender (who was hot) was awesome! The French fries at this place are sick! I got 2 orders they were SO good! And George Washington drank here! It's totally true! I like Stone Street a lot and this is a couple blocks away and it's really worth going to. It's cool to go with friends or to hang out or even on a date. Plus they had this place as the starting point and the after party place for Zombies vs Vampires (zombiesVvampires.com if u want to check it out). Such a cool place! Laid back! Not expensive! Pretty easy to get to on the 1. I wish I knew about it before. You definitely have to try this place out.,-1
111335,69318,54,5,10/14/2011,"I've been here twice now and I'm a huge fan of their food and beer. Their porter is smooth, creamy and very, very tasty. In my opinion, it's better than Guinness. It's worth going there just for the beer. The first time I went was in the evening. Their bar food was exceptional. Their fish and chips are solid, and I liked a mushroom based vegetarian dish. I've forgotten what they're called, but you should definitely their mini-sausages wrapped in dough. In the evenings though, it can get crowded and it's hard to get your waiter's attention. My girlfriend was lucky to win a free lunch there. Their potato and leek soup was sublime and their bangers and mash were very good too. The servers are very nice and helpful and I highly recommend the place.",-1
111644,69460,60,2,1/20/2014,"First off, I would like to be upfront and say that the food is quote good and full of flavor. However the biggest thing that irked me was the size of their platters. The are in that awkward place between the size/price of an appetizer and the size/price of a normal main dish platter. In a regular restaurant I would maybe spend $10 on an appetizer and $20 on a main dish, and this is if I am feeling particularly hungry. Here I literally had to get two ""main dishes"" which amounted to about $35 and still left fairly hungry. If you just want an expensive but flavorful snack, this place is good. But if you are looking for a place to eat this may not be the best value. The interior is also a bit strange. It tried to pull off the ambiance of a restaurant with candles and a darkly lit setting, but the white plastic bar tables are strangely positioned and are cold in nature. Bottom line, the chef is great and the ideas behind the dishes are wonderful. But unless this place changes its value proposition it will not last long. Suggestion - keep th

Here I look at how some of the fake or disingenuous reviews are worded. They all seem real to me, I must be missing something. I also wonder how or why they were rated genuine or not. Was it a person who did this? I could not find the answer on kaggle, I will assume the data is accurate and continue to go with it. 

After doing a little more research, I found out that Yelp also classifies them as 'recommended' or 'not recommended'. https://www.reviewtrackers.com/blog/fake-yelp-reviews/ is the website I discovered this at. Each post is carefully vetted by employees at Yelp. At the current state of that article(Jan 2019), 70% are recommended while 30% are not. Long rants are put in the non recommended, along with those that seem solicitated. They want all reviews to be genuine, without a restaraunt trying to boost their ratings in a disingenuous manner. Yelp also has more information such as which computers the reviews came from and activity of the users.

https://www.youtube.com/watch?v=KdG81QKdga0 is a link to a video where 5 star reviews are sold to businesses. There are Facebook groups where you can go to find people who will boost your score, with the 'elite' reviews being premium since they cannot be removed. This video is from September of 2020. This is why the fake reviews are so hard to spot. They are actual people writing these for money boosting the business rating. This leads to the question, can you also pay a group of people to bring down a competing business?

In [41]:
df.loc[df['Label'] == 1].tail(20)

,User_id,Product_id,Rating,Date,Review,Label
111778,69545,318,5,12/20/2013,"Excellent southern delivery!! I had chicken wings and red velvet waffles with delicious greens. My husband had oxtails and said they were great, albeit a bit salty. I'm definitely ordering again!",1
111779,69546,318,5,11/21/2013,Oh....and BEST WAFFLES in Brooklyn - I MEAN THAT....Try them for breakfast as well,1
111780,69547,318,5,11/20/2013,This place had the best chicken and waffles...as well as other genuine down home cuisine with a nice simple decor that makes a country boy like me feel right at home...I'd your looking for a good meal and to replace the new York attitude for a little southern hospitality check this spot out,1
111781,69548,318,2,11/1/2013,Yelpers you steered me wrong... I got excited reading the reviews because Napoleon is not far from my house. So rushed from work to get her and the food was beyond disappointing. I had the fried fish mac and cheese and the collards. Fish under cooked and bland...I didn't taste an ounce of seasoning I had to use the tarter sauce for that. Mac and cheese I can't put my finger on the cheese but it taste powdery like Â a combination of kraft mac and cheese with real cheese added for good measure. Colards wouldn't have been bad if they weren't salty The young lady said she knew I would come back...I think not. I could have had end if the day food and theres a chance it may be better during regular hours...but I don't want to find out.,1
111782,69549,318,5,10/12/2013,"I don't think that I will be going back for the rest of the year. Here's why: THE FOOD IS TOO GOOD!!!!!!!!! Do you understand what that means? THE F-O-O-D is G-O-O-D! I went into the store because earlier that day, I was CRAVING for some chicken and waffles. I went on yelp because I was going to go to Sweet Chick to test it out, but I work all the way in the Bronx and when I'm off, I don't like traveling and Williamsburg was a stretch from Crown Heights that day. I'm sorry. I see this place on Yelp that's right around the corner so I take the bus and I'm on the hunt. I go into the store that is EXTREMELY clean. I tell the guy that I want the red velvet waffles and chicken wings plus the sweet tea and a side of macaroni and cheese. I have a seat, drooling with anticipation. I almost cried when he set the food in front of me. The red velvet waffle had a sweet cream cheese glaze and it was so good. I asked for syrup on the side, but it wasn't needed. Not too sweet, but it was good. It was just right. BEST WAFFLE I EVER HAD IN MY LIFE! And trust me, I know my way around waffles. Damn a pancakes and DAMN a French toast! I had to order a second waffle. I also had a homemade half and half and It was the best I've ever had in New York State. (Just needs some Jack Daniel's Honey to mix with!) Come to find out that the guy that was serving me was the owner! I LOVE to see a business where the owner is hands on. Just a pleasant and humble person. Respect will get you places money won't. Because the owner is a humble man, his restaurant will flourish. I pray this is so, he deserves it. And he's from Birmingham, AL! It shows in his cooking. Â I wanted to try the banana pudding that was in the window but I didn't have the space. Maybe next year. Fuck am I kidding? I'm cancelling the NYSC membership and I'm at Napoleon's TOMORROW!!!!!! Love and life.",1
111783,69550,318,5,10/6/2013,"Red velvet waffles with chicken wings. Don't order anything else, it's incredible!",1
111784,16438,318,3,9/15/2013,"I look forward to updating this review to give it a better rating because the food (for the most part) was good! Unfortunately, it took them 2 hours to deliver the food when they originally estimated 35-45 mins, that knocked down the 4th star. The chicken was DARN good! We'll skip the fries next time, they were super soggy and greasy with a slice of american cheese melted on top. Unfortunately we couldn't tell if the red velvet waffle was good or not, it sweated o

Here I look at how the real reviews are worded. I find them worded similarly to the fake reviews so now I am certain I must be missing something. Hopefully the model can pick up on something I can't.